<a href="https://colab.research.google.com/github/Bubukisapisa/ML_education/blob/main/%22HW_2_2_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

In [7]:
segm_df = pd.read_csv('drive/MyDrive/ML for people/customer_segmentation_train.csv')

In [8]:
segm_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [9]:
from os import replace
segm_df = segm_df.drop('ID', axis=1)

In [10]:
segm_df.nunique()

,0
Gender,2
Ever_Married,2
Age,67
Graduated,2
Profession,9
Work_Experience,15
Spending_Score,3
Family_Size,9
Var_1,7
Segmentation,4


In [11]:
segm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           8068 non-null   object 
 1   Ever_Married     7928 non-null   object 
 2   Age              8068 non-null   int64  
 3   Graduated        7990 non-null   object 
 4   Profession       7944 non-null   object 
 5   Work_Experience  7239 non-null   float64
 6   Spending_Score   8068 non-null   object 
 7   Family_Size      7733 non-null   float64
 8   Var_1            7992 non-null   object 
 9   Segmentation     8068 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 630.4+ KB


In [12]:
segm_df['Ever_Married'].value_counts()

,count
Ever_Married,
Yes,4643
No,3285


In [13]:
segm_df['Graduated'].value_counts()

,count
Graduated,
Yes,4968
No,3022


In [14]:
segm_df['Family_Size'].value_counts()

,count
Family_Size,
2.0,2390
3.0,1497
1.0,1453
4.0,1379
5.0,612
6.0,212
7.0,96
8.0,50
9.0,44


In [15]:
segm_df.Profession.unique()

array(['Healthcare', 'Engineer', 'Lawyer', 'Entertainment', 'Artist',
       'Executive', 'Doctor', 'Homemaker', 'Marketing', nan], dtype=object)

In [16]:
#Заповнення категоріальних даних
segm_df['Ever_Married'] = segm_df['Ever_Married'].fillna(segm_df['Ever_Married'].mode()[0])
segm_df['Graduated'] = segm_df['Graduated'].fillna(segm_df['Graduated'].mode()[0])
segm_df['Profession'] = segm_df['Profession'].fillna('Unknown')
segm_df['Var_1'] = segm_df['Var_1'].fillna('Unknown')

#Заповнення числових даних
segm_df['Family_Size'] = segm_df['Family_Size'].fillna(segm_df['Family_Size'].mode()[0])
segm_df['Work_Experience'] = segm_df['Work_Experience'].fillna(segm_df['Work_Experience'].median())

In [61]:
num_cols = segm_df.select_dtypes('number').columns[1:]
non_binary_numeric = [col for col in num_cols if segm_df[col].nunique() > 2]
cat_cols = segm_df.select_dtypes('object').columns[:-1]
cat_cols

Index(['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score',
       'Var_1'],
      dtype='object')

In [18]:
#Трансформуєм числові дані
num_scale = StandardScaler()
num_scale.fit(segm_df[non_binary_numeric])
segm_df[non_binary_numeric] = num_scale.transform(segm_df[non_binary_numeric])

# cat_trans = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# cat_trans.fit(segm_df[cat_cols])

# encoded_cols = list(cat_trans.get_feature_names_out(['Profession', 'Spending_Score', 'Var_1']))
# segm_df[encoded_cols] = cat_trans.transform(segm_df[cat_cols])

In [19]:
segm_df

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,Male,No,22,No,Healthcare,-0.451136,Low,0.785536,Cat_4,D
1,Female,Yes,38,Yes,Engineer,-0.451136,Average,0.122735,Cat_4,A
2,Female,Yes,67,Yes,Engineer,-0.451136,Low,-1.202868,Cat_6,B
3,Male,Yes,67,Yes,Lawyer,-0.757410,High,-0.540066,Cat_6,B
4,Female,Yes,40,Yes,Entertainment,-0.451136,High,2.111139,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...
8063,Male,No,22,No,Unknown,-0.757410,Low,2.773940,Cat_1,D
8064,Male,No,35,No,Executive,0.161413,Low,0.785536,Cat_4,D
8065,Female,No,33,Yes,Healthcare,-0.451136,Low,-1.202868,Cat_6,D
8066,Female,No,27,Yes,Healthcare,-0.451136,Low,0.785536,Cat_6,B


In [20]:
X_train, X_test, y_train, y_test = train_test_split(segm_df.drop('Segmentation', axis=1),
                                                    segm_df['Segmentation'],
                                                    test_size = 0.2,
                                                    random_state = 42,
                                                    stratify = segm_df['Segmentation']
                                                    )

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [21]:
X_train

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
917,Female,No,32,Yes,Artist,1.999059,Low,-1.202868,Cat_6
3398,Male,Yes,72,Yes,Entertainment,-0.451136,Average,-0.540066,Cat_6
2045,Female,No,33,Yes,Entertainment,-0.451136,Low,0.785536,Cat_6
8060,Female,Yes,48,Yes,Artist,-0.757410,Average,2.111139,Cat_6
4604,Female,Yes,28,No,Doctor,1.999059,Low,-1.202868,Cat_7
...,...,...,...,...,...,...,...,...,...
3822,Female,No,27,No,Homemaker,1.692785,Low,-1.202868,Cat_6
5864,Male,No,37,Yes,Healthcare,-0.757410,Low,-0.540066,Cat_6
3589,Female,No,27,No,Engineer,1.080236,Low,2.111139,Cat_4
1489,Male,No,26,No,Artist,-0.757410,Low,-0.540066,Cat_6


In [22]:
cat_cols_index = [segm_df.columns.get_loc(c) for c in cat_cols]

smote = SMOTE(random_state=0)
smotenc = SMOTENC(categorical_features=cat_cols_index, random_state=42)
smotetomek = SMOTETomek(random_state=0)

X_train_smote, y_train_smote = smote.fit_resample(X_train[num_cols], y_train)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train, y_train)
X_train_smotetomek, y_train_smotetomek = smotetomek.fit_resample(X_train[num_cols], y_train)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [23]:
X_train_orig,  X_test_orig, y_train_orig, y_test_orig = X_train, X_test, y_train, y_test

#Зміна категоріальних бінарних даних
X_train_orig['Ever_Married'] = X_train_orig['Ever_Married'].replace({'Yes': 1, 'No': 0})
X_train_orig['Graduated'] = X_train_orig['Graduated'].replace({'Yes': 1, 'No': 0})
X_train_orig['Gender'] = X_train_orig['Gender'].replace({'Male': 1, 'Female': 0})

cat_cols_orig = X_train_orig.select_dtypes('object').columns
print(cat_cols_orig)
cat_trans = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_trans.fit(X_train_orig[cat_cols_orig])

X_test_orig['Ever_Married'] = X_test_orig['Ever_Married'].replace({'Yes': 1, 'No': 0})
X_test_orig['Graduated'] = X_test_orig['Graduated'].replace({'Yes': 1, 'No': 0})
X_test_orig['Gender'] = X_test_orig['Gender'].replace({'Male': 1, 'Female': 0})

cat_trans = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_trans.fit(X_test_orig[cat_cols_orig])

encoded_cols = list(cat_trans.get_feature_names_out())
X_train_orig[encoded_cols] = cat_trans.transform(X_train_orig[cat_cols_orig])
X_test_orig[encoded_cols] = cat_trans.transform(X_test_orig[cat_cols_orig])

Index(['Profession', 'Spending_Score', 'Var_1'], dtype='object')


/tmp/ipython-input-1768951134.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train_orig['Ever_Married'] = X_train_orig['Ever_Married'].replace({'Yes': 1, 'No': 0})
/tmp/ipython-input-1768951134.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train_orig['Graduated'] = X_train_orig['Graduated'].replace({'Yes': 1, 'No': 0})
/tmp/ipython-input-1768951134.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_ob

In [37]:
X_train_orig = X_train_orig.drop(['Profession', 'Spending_Score',
       'Var_1'], axis=1)
X_test_orig = X_test_orig.drop(['Profession', 'Spending_Score',
       'Var_1'], axis=1)

In [52]:
num_cols_2 = X_train_orig.select_dtypes('number').columns
num_cols_2

Index(['Gender', 'Ever_Married', 'Age', 'Graduated', 'Work_Experience',
       'Family_Size', 'Profession_Artist', 'Profession_Doctor',
       'Profession_Engineer', 'Profession_Entertainment',
       'Profession_Executive', 'Profession_Healthcare', 'Profession_Homemaker',
       'Profession_Lawyer', 'Profession_Marketing', 'Profession_Unknown',
       'Spending_Score_Average', 'Spending_Score_High', 'Spending_Score_Low',
       'Var_1_Cat_1', 'Var_1_Cat_2', 'Var_1_Cat_3', 'Var_1_Cat_4',
       'Var_1_Cat_5', 'Var_1_Cat_6', 'Var_1_Cat_7', 'Var_1_Unknown'],
      dtype='object')

In [53]:
#Логістична регресія на оригінальних даних
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_train_orig, y_train)
ovr_predictions = ovr_model.predict(X_test_orig)

In [62]:
#Логістична регресія на SMOTE даних
log_reg_smote = LogisticRegression(solver='liblinear')
smote_model_smote = OneVsRestClassifier(log_reg_smote)
smote_model_smote.fit(X_train_smote, y_train_smote)
smote_predictions = smote_model_smote.predict(X_test[num_cols])

In [48]:
#Логістична регресія на SMOTETOMEK даних
log_reg_smotetomek = LogisticRegression(solver='liblinear')
smotetomek_model = OneVsRestClassifier(log_reg_smotetomek)
smotetomek_model.fit(X_train_smotetomek, y_train_smotetomek)
smotetomek_predictions = smotetomek_model.predict(X_test[num_cols])

In [64]:
#Препроцесинг для категоріальних даних згенерованих SMOTENC
#Зміна категоріальних бінарних даних
X_train_smotenc['Ever_Married'] = X_train_smotenc['Ever_Married'].replace({'Yes': 1, 'No': 0})
X_train_smotenc['Graduated'] = X_train_smotenc['Graduated'].replace({'Yes': 1, 'No': 0})
X_train_smotenc['Gender'] = X_train_smotenc['Gender'].replace({'Male': 1, 'Female': 0})

cat_cols_orig = X_train_smotenc.select_dtypes('object').columns
print(cat_cols_orig)
cat_trans = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_trans.fit(X_train_smotenc[cat_cols_orig])

encoded_cols = list(cat_trans.get_feature_names_out())
X_train_smotenc[encoded_cols] = cat_trans.transform(X_train_smotenc[cat_cols_orig])

X_train_smotenc = X_train_smotenc.drop(['Profession', 'Spending_Score',
       'Var_1'], axis=1)

Index(['Profession', 'Spending_Score', 'Var_1'], dtype='object')


In [66]:
#Логістична регресія на SMOTENC даних
log_reg_smotenc = LogisticRegression(solver='liblinear')
smotenc_model = OneVsRestClassifier(log_reg_smotenc)
smotenc_model.fit(X_train_smotenc, y_train_smotenc)
smotenc_predictions = smotenc_model.predict(X_test_orig)

In [67]:
#Оцінка моделей
print(classification_report(y_test, ovr_predictions))

              precision    recall  f1-score   support

           A       0.42      0.46      0.44       394
           B       0.41      0.17      0.24       372
           C       0.49      0.62      0.55       394
           D       0.65      0.76      0.70       454

    accuracy                           0.52      1614
   macro avg       0.49      0.50      0.48      1614
weighted avg       0.50      0.52      0.49      1614



In [68]:
#Оцінка моделей
print(classification_report(y_test, smote_predictions))

              precision    recall  f1-score   support

           A       0.34      0.57      0.43       394
           B       0.28      0.13      0.18       372
           C       0.28      0.29      0.28       394
           D       0.40      0.32      0.35       454

    accuracy                           0.33      1614
   macro avg       0.32      0.33      0.31      1614
weighted avg       0.33      0.33      0.31      1614



In [69]:
#Оцінка моделей
print(classification_report(y_test, smotetomek_predictions))

              precision    recall  f1-score   support

           A       0.34      0.57      0.43       394
           B       0.28      0.13      0.18       372
           C       0.28      0.29      0.28       394
           D       0.40      0.32      0.35       454

    accuracy                           0.33      1614
   macro avg       0.32      0.33      0.31      1614
weighted avg       0.33      0.33      0.31      1614



In [70]:
#Оцінка моделей
print(classification_report(y_test, smotenc_predictions))

              precision    recall  f1-score   support

           A       0.42      0.49      0.45       394
           B       0.39      0.24      0.30       372
           C       0.51      0.57      0.54       394
           D       0.67      0.71      0.69       454

    accuracy                           0.51      1614
   macro avg       0.50      0.50      0.49      1614
weighted avg       0.50      0.51      0.50      1614



Якщо говорити про те яка модель найкраще впоралась то виходить що звичайна логістична регресія та smotenc найкраще впорались. Але загалом все одно всюди вийшов незадовлільний результат. Чому так - не розумію.
Для оцінки, напевно, використовував би macro avg так як кожен клас A, B, C, D  для нас рівнозначний